This notebook will show how to use the LSST Simulations code which can connect coordinates on the sky (RA, Dec) with coordinates on the LSST camera (pixels).  <b>Note: Before running this notebook, you must install and set up the packages ```obs_lsstSim```, which actually contains the representation of the LSST camera, and ```sims_coordUtils```, which contains the code wrapping obs_lsstSim.  You can get both of these by simply installing and setting up ```lsst_sims```.</b>

First, let's import the representation of the LSST Camera and instantiate it.

In [ ]:
from __future__ import print_function
from lsst.obs.lsstSim import LsstSimMapper
camera = LsstSimMapper().camera

The global object ```camera``` now contains a representation of the LSST camera.  ```camera``` is a dict-like object containing ```detector``` objects that characterize the CCDs in the LSST focal plane.  They can be iterated over.

In [ ]:
from lsst.afw.cameraGeom import DetectorType

ccd_type_dict = {}
ccd_type_dict[DetectorType.SCIENCE] = 'science'
ccd_type_dict[DetectorType.WAVEFRONT] = 'wavefront'
ccd_type_dict[DetectorType.GUIDER] = 'guider'
ccd_type_dict[DetectorType.FOCUS] = 'focus'

for det in camera:
    print('%s %s' % (det.getName(), ccd_type_dict[det.getType()]))

As you can see above, in addition to containing a grid of pixels, each detector also has a name and a type.  You can refer to detectors by name

In [ ]:
tt = camera['R:0,4 S:1,0'].getType()
print(ccd_type_dict[tt])

The 'R:X,Y S:I,J' is a way of referring to how the chips are layed out in rafts on the camera.  For more information, see https://confluence.lsstcorp.org/display/LSWUG/Representation+of+a+Camera

The functionality of the ```camera``` object is all defined in the package ```afw.cameraGeom```.  Users wishing to interact directly with the ```camera``` object should consult the documentation and examples provided with that package.  The LSST Simulations stack provides methods that allow you to work with ```camera``` without having to call its methods directly.  Those are the subject of this notebook.

## Finding which chip an object lands on

```sims_coordUtils``` provides a method ```chipNameFromRaDec``` which allows you to specify a position on the sky and get back the name of the chip on which that position falls.  In order to use this method, you must first create an ```ObservationMetaData``` characterizing the direction that the telescope is pointing and how it is rotated with respect to the sky.

In [ ]:
from lsst.sims.utils import ObservationMetaData
obs = ObservationMetaData(pointingRA=123.0, pointingDec=-35.6,
                          rotSkyPos=29.0, mjd=59580.0)

<b>Note:</b> This cell will raise an ```ErfaWarning```.  This is harmless.  We are asking ```astropy.Time``` to convert between UTC (universal coordinate time) and TAI (international atomic time) in the future (when LSST goes on-sky), which is technically impossible, given that one cannot predict when leap seconds will be added.

In [ ]:
import numpy as np
from lsst.sims.coordUtils import chipNameFromRaDec
chipName = chipNameFromRaDec(121.0, -35.2, 
                             camera=camera, obs_metadata=obs)
print(chipName)

You can also pass a numpy array of RA, Dec and get back a list of chip names

In [ ]:
chipNameList = chipNameFromRaDec(np.array([121.0, 121.7, 124.0]),
                                 np.array([-35.2, -35.1, -34.0]),
                                 camera=camera, obs_metadata=obs)
print(chipNameList)

There are also methods to take (RA, Dec) positions on the sky and transform them into pixel coordinates on the camera.  Note that pixel coordinates are defined on each individual chip (i.e. each chip has its own (0, 0)).  The ```pixelCoordsFromRaDec``` method can accept the kwarg ```chipName``` to specify on which chip you are asking for coordinates.  If you do not specify ```chipName```, the method will take your given (RA, Dec) positions, find the chips they land on, and return the pixel coordinates on those chips.

In [ ]:
from lsst.sims.coordUtils import pixelCoordsFromRaDec

# first let's just calculate the pixel coordinates on the
# chips where the (RA, Dec) points naturally fall.

xPix, yPix = pixelCoordsFromRaDec(np.array([121.0, 121.7, 124.0]),
                                 np.array([-35.2, -35.1, -34.0]),
                                 camera=camera, obs_metadata=obs)

for xx, yy in zip(xPix, yPix):
    print(xx, yy)

In [ ]:
# Now let us specify a specific chip.  In this case,
# pixelCoordsFromRaDec will return the pixel coordinates of
# each (RA, Dec) point as if it fell on the R:0,2 S:2,2 chip

xPix, yPix = pixelCoordsFromRaDec(np.array([121.0, 121.7, 124.0]),
                                  np.array([-35.2, -35.1, -34.0]),
                                  chipName='R:0,2 S:2,2',
                                  camera=camera, obs_metadata=obs)

for xx, yy in zip (xPix, yPix):
    print(xx, yy)

Note that if you have already calculated the names of the chips on which your (RA, Dec) points fall, you can speed up ```pixelCoordsFromRaDec``` by passing those in as a kwarg (that way, ```pixelCoordsFromRaDec``` will not have to duplicate the call to ```chipNameFromRaDec``` that you already made)

In [ ]:
raList = np.array([121.0, 121.7, 124.0])
decList = np.array([-35.2, -35.1, -34.0])

chipNameList = chipNameFromRaDec(raList, decList,
                                 camera=camera, obs_metadata=obs)

xPix, yPix = pixelCoordsFromRaDec(raList, decList, chipName=chipNameList,
                                 camera=camera, obs_metadata=obs)

for xx, yy in zip(xPix, yPix):
    print(xx, yy)

## Finding the bounds of a chip

There are also methods to get the coordinate values (in pixels and in RA, Dec) of a specific chip.  These methods output lists of tuples in which each tuple represents the coordinates of one of the corners of the chip.

In [ ]:
from lsst.sims.coordUtils import getCornerPixels, getCornerRaDec

cpix = getCornerPixels('R:3,4 S:0,1', camera)
print('Corner pixels ',cpix,'\n')

cCoord = getCornerRaDec('R:3,4 S:0,1', camera, obs_metadata=obs)
print('Corner (RA, Dec) pairs: ',cCoord)

<b>Note:</b> Because of the rotation angle between the camera and the sky, the RA, Dec corner values will not be in any order relative to ```(min, max)```.